# Imports, Installs, and Set Up

In [ ]:
%%bash
### No Need to Modify ###
pip install gdown --upgrade --no-cache-dir --quiet &&
gdown --id 11S8ROPmgjhIakLOcBhz-SJ2q0-d3wTrV &&
tar -xf 6-homologs-genes.tar.gz


In [ ]:
import os
os.chdir('6-homologs-genes')

In [ ]:
%env PYTHONPATH=

In [ ]:
%%bash
### No Need to Modify ###
MINICONDA_INSTALLER_SCRIPT=Miniconda3-latest-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT --quiet
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

In [ ]:
%%bash
### No Need to Modify ###
conda install -y -q -c bioconda mafft=7.487

In [ ]:
%%bash
### No Need to Modify ###
conda install -y -q -c bioconda blast

In [ ]:
%%bash
### No Need to Modify ###
conda install -y -q -c bioconda  clipkit

In [ ]:
%%bash
### No Need to Modify ###
conda install -y -q -c bioconda iqtree

# Run Pipeline

In [ ]:
#000-group-basal-metazoans

# Nematostella vectensis
# Morbakka virulenta
# Hydra vulgaris
# Thelohanellus kitauei
# Rhopilema esculentum
# Hormiphora californensis
# Mnemiopsis leidyi
# Trichoplax adhaerens
# Ephydatia muelleri
# Amphimedon queenslandica
# Monosiga brevicollis
# Capsaspora owczarzaki
# Sphaeroforma arctica
# Homo sapiens
# Drosophila melanogaster
# Caenorhabditis elegans


In [ ]:
%%bash
#001-ls-blastdbs

mkdir output || echo ""
echo
ls -1 4-projectdb-projectdb/projectdb-fastas-provided/*aa > output/1-list-projectdb-blastdbs
ls -1 source-rgs/rgs5-piezo-full_length-03dec2021.fasta > output/1-list-rgs


In [ ]:
#002-python-command-blastp

# input_list
# localdb/projectdb-Metazoa-Nematoda-Chromadorea-Rhabditida-Onchocercidae-Brugia-malayi-6279-Bmal-4.0.fasta

input_list = open( 'output/1-list-projectdb-blastdbs', 'r' )
input_rgs = open( 'output/1-list-rgs', 'r' )
output_command = open( '003-blastp_X_projectDB', 'w' )

for next_rgs in input_rgs:

    rgs_fasta = next_rgs[ :-1 ] # ends up assigning to pore region in this case

for next_line in input_list:

    db_path = next_line[ :-1 ]
    info = next_line[ :-1 ].split( '-' )
    genus = info[ 10 ]
    species = info[ 11 ]
    gspp = genus + '-' + species
    
    output = 'blastp -db ' + db_path  + ' -outfmt 6 -out output/3-blast-report-rgs_X_projectdb-' + gspp + ' -max_hsps 1 -query ' + rgs_fasta + ' -evalue 1e-3\n'
    output_command.write( output )
    
output = 'echo\n'
output_command.write( output )

input_list.close()
input_rgs.close()
output_command.close()


In [ ]:
%%bash
#003-blastp_X_projectDB
source $HOME/miniconda3/bin/activate
bash 003-blastp_X_projectDB


In [ ]:
%%bash
#004-ls-reports-and-fastas

ls output/3-blast-report-rgs_X_projectdb-* > output/4-list-reports
ls 4-projectdb-projectdb/projectdb-fastas-provided/*aa > output/4-list-fastas


In [ ]:
#005-python-gene-set-fasta

# rgs73-worm-TRPA_Cele_TRPA-TRPA2_1_481-uniprotQ21517-extraction_51_366   Metazoa-Chordata-Coelacanthimorpha-Coelacanthiformes-Coelacanthidae-Latimeria-chalumnae-pdb0000292542   26.036  169     98      4       140     299     846     996     3.39e-05        44.4

input_fastas = open( 'output/4-list-fastas', 'r' )
input_reports = open( 'output/4-list-reports', 'r' )
output_geneset_gspp = open( 'output/5-blastp-hits.fasta', 'w')
output_geneset_gspp_sub = open( 'output/5-blastp-pore-region-hits.fasta', 'w')

gene_seq = {}
gene_coordinates = {}
for next_fasta in input_fastas:
    
    fasta = next_fasta[ :-1 ]
    input_fasta = open( fasta, 'r' )
    
    for next_line in input_fasta:

        if next_line[ 0 ] == '>':
            
            header = next_line[ 1:-1 ]
            gene_seq[ header ] = ''

        else:

            sequence = next_line[ :-1 ]
            gene_seq[ header ] = gene_seq[ header ] + sequence

    input_fasta.close()

for next_report in input_reports:

    all_hits = []
    report = next_report[ :-1 ]
    input_report = open( report, 'r' )
 
    for next_line in input_report:

        report_info = next_line.split( '\t' )
        gene_id = report_info[ 1 ]
        all_hits.append( gene_id )
        coordinate_1 = int( report_info[ 8 ] ) - 1
        coordinate_2 = int( report_info[ 9 ] ) - 1
        gene_coordinates[ gene_id ] = ( coordinate_1, coordinate_2 )
        
    unique_hits = list( set( all_hits ) )

    for next_hit in unique_hits:
        
        gene_id = next_hit
        sequence = gene_seq[ gene_id ]
        coordinate_1 = gene_coordinates[ gene_id ][ 0 ]
        coordinate_2 = gene_coordinates[ gene_id ][ 1 ]
        subsequence = sequence[ coordinate_1 : coordinate_2 ] 

        output = '>' + gene_id + '\n' + sequence + '\n'
        output_geneset_gspp.write( output )
        
        output_sub = '>' + gene_id + '\n' + subsequence + '\n'
        output_geneset_gspp_sub.write( output_sub )

    input_report.close()

input_fastas.close()
input_reports.close()
output_geneset_gspp.close()
output_geneset_gspp_sub.close()


In [ ]:
%%bash
#006-blastp-rgs_X_rgs-genomes
source $HOME/miniconda3/bin/activate

blastp -db 4-projectdb-projectdb/projectdb-fastas-provided/projectdb-Metazoa-Chordata-Mammalia-Primates-Hominidae-Homo-sapiens*aa -query source-rgs/rgs5-piezo-full_length-03dec2021.fasta -out output/6-blast-report-rgs5-piezo-family_X_human-genome -outfmt 6  -max_hsps 1

blastp -db 4-projectdb-projectdb/projectdb-fastas-provided/projectdb-Metazoa-Arthropoda-Insecta-Diptera-Drosophilidae-Drosophila-melanogaster*aa -query source-rgs/rgs5-piezo-full_length-03dec2021.fasta -out output/6-blast-report-rgs5-piezo-family_X_fly-genome -outfmt 6  -max_hsps 1

blastp -db 4-projectdb-projectdb/projectdb-fastas-provided/projectdb-Metazoa-Nematoda-Chromadorea-Rhabditida-Rhabditidae-Caenorhabditis-elegans*aa -query source-rgs/rgs5-piezo-full_length-03dec2021.fasta -out output/6-blast-report-rgs5-piezo-family_X_worm-genome -outfmt 6  -max_hsps 1


In [ ]:
%%bash
#007-ls-rgs-reports-fastas

ls output/6*genome* > output/7-list-reports

ls 4-projectdb-projectdb/projectdb-fastas-provided/*Homo-sapiens*aa > output/7-list-rgs-projectdb-fastas
ls 4-projectdb-projectdb/projectdb-fastas-provided/*Drosophila-melanogaster*aa >> output/7-list-rgs-projectdb-fastas
ls 4-projectdb-projectdb/projectdb-fastas-provided/*Caenorhabditis-elegans*aa >> output/7-list-rgs-projectdb-fastas


In [ ]:
#008-python-update-reference-genomes


##### USER INPUT
input_reports = open( 'output/7-list-reports', 'r' )
input_fastas = open( 'output/7-list-rgs-projectdb-fastas', 'r' )
input_rgsfasta = open( 'source-rgs/rgs5-piezo-full_length-03dec2021.fasta', 'r' )
output_map = open( 'output/8-map-source-to-reference-identifiers', 'a' )

model_species = [ 'human','mouse','fly','worm','anemone' ]

###### BEING SCRIPT 
# read in reference gene header identifier and sequence into dictionary
rgs_seq = {}
for next_line in input_rgsfasta:

    if next_line[ 0 ] == '>':

        identifier = next_line[ 1:-1 ].split( ' ' )[ 0 ]
        rgs_seq[ identifier ] = ''

    else:

        rgs_seq[ identifier ] = rgs_seq[ identifier ] + next_line[ :-1 ]

# read rgs query and rgs genome top hit into dictionary
gengene_refgene = {}
rgs_genes = []
gengenes = []

for next_report in input_reports:
    
    input_report = open( next_report[ :-1 ], 'r' )
    
    model_name = ""
    for model in model_species:
        if model in next_report:
            model_name = model
            break

    for next_hit in input_report:

        info = next_hit.split( '\t' )
        refgene = info[ 0 ]
        gengene = info[ 1 ]
        name = refgene.split( '-' )[ 1 ]
        
        if name == model_name:

            if (refgene in rgs_genes) or (gengene in gengenes):

                pass

            else:

                gengene_refgene[ gengene ] = refgene
                rgs_genes.append( refgene )
                gengenes.append( gengene )

        else:

            pass
            
    input_report.close()


# read in RGS genome and replace rgs genes (top hit in blast of rgs _X_ rgs genome) with rgs header and sequence
header_seq = {}
for next_fasta in input_fastas:

    input_fasta = open( next_fasta[ :-1 ], 'r' )
    output_name = 'output/8-' + next_fasta[ :-1 ].split( '/' )[ -1 ] + '-rgs'
    output_fasta = open( output_name, 'w' )

    for next_line in input_fasta:

        if next_line[ 0 ] == '>':

            count = 0
            
            header_info = next_line[ 1:-1 ]

            if header_info in gengene_refgene.keys():
                
                count = 1
                gengene = header_info
                refgene = gengene_refgene[ gengene ]
                
                header = '>' + refgene + '\n'
                output_fasta.write( header )

                
                output = gengene + '\t' + refgene + '\n'
                output_map.write( output )                
                
            else:

                header = next_line
                output_fasta.write( header )
                
        else:

            if count == 0:

                sequence = next_line
                output_fasta.write( sequence )

            else:

                sequence = rgs_seq[ refgene ] + '\n'
                output_fasta.write( sequence )

    input_fasta.close()
    output_fasta.close()

input_reports.close()
input_fastas.close()
input_rgsfasta.close()
output_map.close()


In [ ]:
%%bash
#009-ls-reference-genome-fastas

cat output/8*rgs > output/9-rgs-all-genomes-combined.fasta-rgs
ls -1 output/9*rgs > output/9-list-RGS-header-reference-genome-fastas



In [ ]:
#010-python-command-makeblastdb

input_list = open( 'output/9-list-RGS-header-reference-genome-fastas', 'r' )
output_makedb = open( '011-blastp-makedb', 'w' )

for next_fasta in input_list:

    fasta_path = next_fasta[ :-1 ]
    fasta = fasta_path.split( '/' )[ -1 ]
    
    db_name = fasta.split( '.' )[ 0 ] + '-AA'
    
    command = 'makeblastdb -in '  + fasta_path + ' -parse_seqids -dbtype prot'
    output = command + '\n'
    output_makedb.write( output )

input_list.close()
output_makedb.close()


In [ ]:
%%bash
#011-blastp-makedb
source $HOME/miniconda3/bin/activate

bash 011-blastp-makedb

In [ ]:
%%bash
#012-ls-blastp-rgs-genomes

ls -1 output/9-rgs-all-genomes-combined.fasta-rgs > output/12-list-blastp-annotated-rgs-genomes


In [ ]:
#013-python-command-blastp

# input_list
# 11-projectdb-Homo-sapiens-cgs-AA.dmnd

input_list = open( 'output/12-list-blastp-annotated-rgs-genomes', 'r' )
output_command = open( '014-blastp-hits_X_RGS-genomes', 'w' )

output = '#! /bin/bash\n'
output_command.write( output )

for next_line in input_list:

    db_path = next_line[ :-1 ]
    info = db_path.split( '-' )
    genome = info[ -3 ] + '-' + info[ -2 ]

    output =  'blastp -db ' + db_path  + ' -outfmt 6 -out output/14-blastp-report-blastp_hits_X_RGS-genome-' + genome + '  -max_target_seqs 1 -max_hsps 1 -query output/5-blastp-pore-region-hits.fasta -matrix BLOSUM45 -evalue 1e-3 -num_threads 60 &\n'
 
    output_command.write( output )

input_list.close()
output_command.close()


In [ ]:
%%bash
#014-blastp-hits_X_RGS-genomes
source $HOME/miniconda3/bin/activate

bash 014-blastp-hits_X_RGS-genomes

In [ ]:
%%bash
#015-cat-all-blastp-reports

cat output/14-* > output/15-all-blastp-all-reports


In [ ]:
#016-python-RBF-CGS-each-RGS-genome-fasta


##### USER INPUT

input_fastas = open( 'output/4-list-fastas', 'r' )
input_hits = open( 'output/15-all-blastp-all-reports', 'r' )
input_rgs_ids = open( 'output/8-map-source-to-reference-identifiers', 'r' )

output_fasta = open( 'output/16-CGS-final-sequences-by-blastp-RBF.fasta', 'w' )
output_filtered = open( 'output/16-dropped-queries-no-rgs-top-hit-in-rgs-genome', 'w' )

model_species = [ 'human','mouse','fly','worm','anemone' ]

##### BEGIN SCRIPT

rgs_ids = []

for next_line in input_rgs_ids:

        info = next_line[ :-1 ].split( '\t' )
        projectdb_id = info[ 0 ]
        rgs_id = info[ 1 ]
        rgs_ids.append( rgs_id )
        rgs_ids.append( projectdb_id )

keepers = []
queries = []

for next_hit in input_hits:

        info = next_hit.split( '\t' )
        query = info[ 0 ]
        queries.append( query )
        
        hit = info[ 1 ]        
        hit_info = hit.split( '-' )
        name = hit_info[ 1 ]

        if name in model_species:
                
                # drop RGS genes
                if query in rgs_ids:

                        pass

                else:
                        keepers.append( query )

        else:
                output = query + '\t' + hit + '\n'
                output_filtered.write( output )

# produce post-filtered keepers diamond fasta NO RGS
count = 0

for next_fasta in input_fastas:

        input_fasta = open( next_fasta[ :-1 ], 'r' )

        for next_line in input_fasta:
                
                if next_line[ 0 ] == '>':
                        count = 0
                        identifier = next_line[ 1:-1 ]

                        if identifier in keepers:
                                count = 1
                                header = '>' + identifier + '\n'
                                output_fasta.write( header )
                                
                        else:
                                pass

                else:

                        if count == 0:
                                pass

                        else:

                                sequence = next_line
                                output_fasta.write( sequence )
                                
        input_fasta.close()
            
input_hits.close()
input_fastas.close()
output_fasta.close()
input_rgs_ids.close()


In [ ]:
%%bash
#017-cat-RGS-CGS-fasta

cat source-rgs/rgs5-piezo-full_length-03dec2021.fasta output/16-CGS-final-sequences-by-blastp-RBF.fasta > output/17-AGS_blastp-rgs5-piezo_X_metazoa16_basal_animals_UPDATED.aa


In [ ]:
%%bash
#018-sed-dashes-for-mafft-PACBIO

# clean sequences for other applications
sed -e 's/-/_/g' output/17-AGS_blastp-rgs5-piezo_X_metazoa16_basal_animals_UPDATED.aa > output/18-AGS-rgs5-piezo_X_metazoa16_basal_animals_UPDATED.aa 
sed -i 's/U/X/g' output/18-AGS-rgs5-piezo_X_metazoa16_basal_animals_UPDATED.aa



In [ ]:
%%bash
#019-mafft-pore-sequence-PACBIO
source $HOME/miniconda3/bin/activate

mafft --originalseqonly --maxiterate 1000 --reorder  --bl 45 --thread 40 output/18-AGS-rgs5-piezo_X_metazoa16_basal_animals_UPDATED.aa > output/19-AGS-rgs5-piezo_X_metazoa16_basal_animals_UPDATED.mafft


In [ ]:
%%bash
#020-clipkit-smartgap-pore-sequence_PACBIO
source $HOME/miniconda3/bin/activate

clipkit -l -m smart-gap output/19-AGS-rgs5-piezo_X_metazoa16_basal_animals_UPDATED.mafft -o output/20-AGS-rgs5-piezo_X_metazoa16_basal_animals_UPDATED.clipkit-smartgap


In [ ]:
%%bash
#021-iqtree2-PACBIO
source $HOME/miniconda3/bin/activate

iqtree -s output/20-AGS-rgs5-piezo_X_metazoa16_basal_animals_UPDATED.clipkit-smartgap -m MFP --prefix output/21-AGS-rgs5-piezo_X_metazoa16_basal_animals_UPDATED --rate -B 2000 -alrt 2000 -T AUTO -bnni -safe


In [ ]:
#022-USER_INPUT-broken-and-replacement-sequences

# #repair_1
# Metazoa-Ctenophora-Tentaculata-Lobata-Bolinopsidae-Mnemiopsis-leidyi_PACBIO-Mnemiopsis-leidyi-NonamEVm000133t1
# Metazoa-Ctenophora-Tentaculata-Lobata-Bolinopsidae-Mnemiopsis-leidyi_UPDATED-ML018021a-PA
# Metazoa-Ctenophora-Tentaculata-Lobata-Bolinopsidae-Mnemiopsis-leidyi_UPDATED-ML018022a-PA
# Metazoa-Ctenophora-Tentaculata-Lobata-Bolinopsidae-Mnemiopsis-leidyi_UPDATED-ML018025a-PA
# Metazoa-Ctenophora-Tentaculata-Lobata-Bolinopsidae-Mnemiopsis-leidyi_UPDATED-ML018027a-PA

In [ ]:
#023-python-repair-broken-gene-models-with-pacbio

input_repair_ids = open( '022-USER_INPUT-broken-and-replacement-sequences', 'r' )
input_ags_broken = open( 'output/17-AGS_blastp-rgs5-piezo_X_metazoa16_basal_animals_UPDATED.aa', 'r' )
output_ags = open( 'output/23-AGS_blastp-rgs5-piezo_X_metazoa16_basal_animals_REPAIRED.aa', 'w' )
output_dropped = open( 'output/23-dropped-sequences.fasta', 'w' )

pacbio_ids = []
broken_ids = []
count = 0
for next_line in input_repair_ids:

    if next_line[ 0 ] == '#':
        count = 1

    elif count == 1:
        pacbio_id = next_line[ :-1 ]
        pacbio_ids.append( pacbio_id )
        count = 0
        
    else:
        broken_id = next_line[ :-1 ]
        broken_ids.append( broken_id )

ags_initial_seq = {}
for next_line in input_ags_broken:

    if next_line[ 0 ] == '>':

        gene_id = next_line[ 1:-1 ]
        ags_initial_seq[ gene_id ] = ''

    else:

        sequence = next_line[ :-1 ]
        ags_initial_seq[ gene_id ] = ags_initial_seq[ gene_id ] + sequence

for next_id in ags_initial_seq.keys():

    sequence = ags_initial_seq[ next_id ]

    if next_id in broken_ids:
        output = '>' + next_id + '\n' + sequence + '\n'
        output_dropped.write( output )

    elif next_id in pacbio_ids:
        output = '>' + next_id + '\n' + sequence + '\n'
        output_ags.write( output )

    elif len( next_id.split( 'Mnemiopsis' ) ) > 1:

        if len( next_id.split( 'UPDATED' ) ) > 1:
            output = '>' + next_id + '\n' + sequence + '\n'
            output_ags.write( output )
            
        else:
            output = '>' + next_id + '\n' + sequence + '\n'
            output_dropped.write( output )

    else:
        output = '>' + next_id + '\n' + sequence + '\n'
        output_ags.write( output )

input_repair_ids.close()
input_ags_broken.close()
output_ags.close()
output_dropped.close()
